In [0]:
# def IntialiseEnv():
#     nb_dir = os.path.split(os.getcwd())[0]
#     if nb_dir not in sys.path:
#         sys.path.append(nb_dir)
#     invalid_path='/opt/ros/kinetic/lib/python2.7/dist-packages'
#     if invalid_path in sys.path:
#         sys.path.remove(invalid_path)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import sys
sys.path.append('/content/drive/My Drive/DL_PRJ/Traffic_Sign_Recognition_Detection/Source_Code')
  


In [4]:
!pip install import_ipynb

In [0]:
# Libraries
import os


import numpy as np
import math
import random
# import import_ipynb
from sklearn.model_selection import train_test_split
# from Data_Preparation.Data_Preparation import LoadTrainDataSet

In [0]:
import glob
import numpy as np
import cv2
import warnings
import pandas as pd 
import matplotlib.pyplot as plt
# import import_ipynb
import numpy as np
import cv2
import skimage.color
import skimage.filters
import skimage.io
import matplotlib.pyplot as plt
# import import_ipynb
from PIL import Image

In [0]:
## Sharpen Image
def sharpen(img):
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    im = cv2.filter2D(img, -1, kernel)
    plt.imshow(im)
    return im

In [0]:
# Excessive sharpening image
def excessive(img):
    kernel = np.array([[1,1,1], [1,-7,1], [1,1,1]])
    im = cv2.filter2D(img, -1, kernel)
    return im

In [0]:
# Blur of images
def blur(img):
    blur =  cv2.medianBlur(img,5)
    return blur

In [0]:
# Edge Enhancement
def edgeEnhancement(img):
    kernel = np.array([[-1,-1,-1,-1,-1],
                               [-1,2,2,2,-1],
                               [-1,2,8,2,-1],
                               [-2,2,2,2,-1],
                               [-1,-1,-1,-1,-1]])/8.0
    img = cv2.filter2D(img, -1, kernel)
    image=cv2.filter2D(img, -1, kernel)
    return image

In [0]:
# Weight image with another image
def addWeightedImg(img,blur):
    result = cv2.addWeighted(img, 1, blur, -0.5, 0)
    return result

In [0]:
# Contrast Enhancement
def ContrastEnhancement():
    hsvImg = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsvImg[...,1] = hsvImg[...,1]*1.6
    hsvImg[...,2] = hsvImg[...,2]*0.8
    img=cv2.cvtColor(hsvImg,cv2.COLOR_HSV2RGB)

In [0]:
#Edge Detection
def CannyEdgeDetection(img):
    edged=cv2.Canny(img,60,150)

In [0]:
# Find Contours
def FindContours(edged):
    contours, hierarchy=cv2.findContours(edged,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(img,contours,-1,(0,255,0),1)
    ret =40
    img[img>ret]=255
    img[img<=ret]=0
    img =cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

In [0]:
def Convert2Grayscale(img):
    return skimage.color.rgb2gray(img)

In [0]:
def ThresholdSegmentationOtsu(img):
    t = skimage.filters.threshold_otsu(img)
    mask = img > t
    return mask

In [0]:
def RemoveNoiseColouredImg(img):
    dst = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)

In [0]:
def HistogramEqualization(img):
    img_to_yuv = cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
    img_to_yuv[:,:,0] = cv2.equalizeHist(img_to_yuv[:,:,0])
    return cv2.cvtColor(img_to_yuv, cv2.COLOR_YUV2RGB)

In [0]:
def CalculateContrast(img):
    # convert to LAB color space
    lab = cv2.cvtColor(img,cv2.COLOR_RGB2LAB)

    # separate channels
    L,A,B=cv2.split(lab)

    # compute minimum and maximum in 5x5 region using erode and dilate
    kernel = np.ones((5,5),np.uint8)
    min = cv2.erode(L,kernel,iterations = 1)
    max = cv2.dilate(L,kernel,iterations = 1)

    # convert min and max to floats
    min = min.astype(np.float64) 
    max = max.astype(np.float64) 

    # compute local contrast
    contrast = (max-min)/(max+min)

    # get average across whole image
    average_contrast = 100*np.mean(contrast)

    print(str(average_contrast)+"%")

In [0]:
def from_pil(pimg):
    pimg = pimg.convert(mode='RGB')
    nimg = np.asarray(pimg)
    nimg.flags.writeable = True
    return nimg

def to_pil(nimg):
    return Image.fromarray(np.uint8(nimg))

def stretch_pre(nimg):
    """
    from 'Applicability Of White-Balancing Algorithms to Restoring Faded Colour Slides: An Empirical Evaluation'
    """
    nimg = nimg.transpose(2, 0, 1)
    nimg[0] = np.maximum(nimg[0]-nimg[0].min(),0)
    nimg[1] = np.maximum(nimg[1]-nimg[1].min(),0)
    nimg[2] = np.maximum(nimg[2]-nimg[2].min(),0)
    return nimg.transpose(1, 2, 0)

def grey_world(nimg):
    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    mu_g = np.average(nimg[1])
    nimg[0] = np.minimum(nimg[0]*(mu_g/np.average(nimg[0])),255)
    nimg[2] = np.minimum(nimg[2]*(mu_g/np.average(nimg[2])),255)
    return  nimg.transpose(1, 2, 0).astype(np.uint8)

def max_white(nimg):
    if nimg.dtype==np.uint8:
        brightest=float(2**8)
    elif nimg.dtype==np.uint16:
        brightest=float(2**16)
    elif nimg.dtype==np.uint32:
        brightest=float(2**32)
    else:
        brightest==float(2**8)
    nimg = nimg.transpose(2, 0, 1)
    nimg = nimg.astype(np.int32)
    nimg[0] = np.minimum(nimg[0] * (brightest/float(nimg[0].max())),255)
    nimg[1] = np.minimum(nimg[1] * (brightest/float(nimg[1].max())),255)
    nimg[2] = np.minimum(nimg[2] * (brightest/float(nimg[2].max())),255)
    return nimg.transpose(1, 2, 0).astype(np.uint8)

def stretch(nimg):
    return max_white(stretch_pre(nimg))

def retinex(nimg):
    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    mu_g = nimg[1].max()
    nimg[0] = np.minimum(nimg[0]*(mu_g/float(nimg[0].max())),255)
    nimg[2] = np.minimum(nimg[2]*(mu_g/float(nimg[2].max())),255)
    return nimg.transpose(1, 2, 0).astype(np.uint8)

def retinex_adjust(nimg):
    """
    from 'Combining Gray World and Retinex Theory for Automatic White Balance in Digital Photography'
    """
    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    sum_r = np.sum(nimg[0])
    sum_r2 = np.sum(nimg[0]**2)
    max_r = nimg[0].max()
    max_r2 = max_r**2
    sum_g = np.sum(nimg[1])
    max_g = nimg[1].max()
    coefficient = np.linalg.solve(np.array([[sum_r2,sum_r],[max_r2,max_r]]),
                                  np.array([sum_g,max_g]))
    nimg[0] = np.minimum((nimg[0]**2)*coefficient[0] + nimg[0]*coefficient[1],255)
    sum_b = np.sum(nimg[1])
    sum_b2 = np.sum(nimg[1]**2)
    max_b = nimg[1].max()
    max_b2 = max_r**2
    coefficient = np.linalg.solve(np.array([[sum_b2,sum_b],[max_b2,max_b]]),
                                             np.array([sum_g,max_g]))
    nimg[1] = np.minimum((nimg[1]**2)*coefficient[0] + nimg[1]*coefficient[1],255)
    return nimg.transpose(1, 2, 0).astype(np.uint8)

def retinex_with_adjust(nimg):
    return retinex_adjust(retinex(nimg))

def standard_deviation_weighted_grey_world(nimg,subwidth,subheight):
    """
    This function does not work correctly
    """
    nimg = nimg.astype(np.uint32)
    height, width,ch = nimg.shape
    strides = nimg.itemsize*np.array([width*subheight,subwidth,width,3,1])
    shape = (height/subheight, width/subwidth, subheight, subwidth,3)
    blocks = np.lib.stride_tricks.as_strided(nimg, shape=shape, strides=strides)
    y,x = blocks.shape[:2]
    std_r = np.zeros([y,x],dtype=np.float16)
    std_g = np.zeros([y,x],dtype=np.float16)
    std_b = np.zeros([y,x],dtype=np.float16)
    std_r_sum = 0.0
    std_g_sum = 0.0
    std_b_sum = 0.0
    for i in xrange(y):
        for j in xrange(x):
            subblock = blocks[i,j]
            subb = subblock.transpose(2, 0, 1)
            std_r[i,j]=np.std(subb[0])
            std_g[i,j]=np.std(subb[1])
            std_b[i,j]=np.std(subb[2])
            std_r_sum += std_r[i,j]
            std_g_sum += std_g[i,j]
            std_b_sum += std_b[i,j]
    sdwa_r = 0.0
    sdwa_g = 0.0
    sdwa_b = 0.0
    for i in xrange(y):
        for j in xrange(x):
            subblock = blocks[i,j]
            subb = subblock.transpose(2, 0, 1)
            mean_r=np.mean(subb[0])
            mean_g=np.mean(subb[1])
            mean_b=np.mean(subb[2])
            sdwa_r += (std_r[i,j]/std_r_sum)*mean_r
            sdwa_g += (std_g[i,j]/std_g_sum)*mean_g
            sdwa_b += (std_b[i,j]/std_b_sum)*mean_b
    sdwa_avg = (sdwa_r+sdwa_g+sdwa_b)/3
    gain_r = sdwa_avg/sdwa_r
    gain_g = sdwa_avg/sdwa_g
    gain_b = sdwa_avg/sdwa_b
    nimg = nimg.transpose(2, 0, 1)
    nimg[0] = np.minimum(nimg[0]*gain_r,255)
    nimg[1] = np.minimum(nimg[1]*gain_g,255)
    nimg[2] = np.minimum(nimg[2]*gain_b,255)
    return nimg.transpose(1, 2, 0).astype(np.uint8)


In [0]:
# Final Feature Extraction Algo
# See if there any room for improvements
# See if colour consistency make any changes for real
def FeatureExtraction(img):
    pil_image=to_pil(retinex((img)))
    open_cv_image = np.array(pil_image) 
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    hist_equalization_result = HistogramEqualization(open_cv_image)
    edgeEnhanced=edgeEnhancement(hist_equalization_result)
    gray= Convert2Grayscale(edgeEnhanced)
    return gray

In [0]:
def ProposedSegmentationAlgoWithDisplay(path): 
    img=loadSampleImg(path)
    pil_image=to_pil(retinex((img)))
    open_cv_image = np.array(pil_image) 
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    hist_equalization_result = HistogramEqualization(open_cv_image)
    edgeEnhanced=edgeEnhancement(hist_equalization_result)
    gray= Convert2Grayscale(edgeEnhanced)
    fig, ax = plt.subplots(nrows=1, ncols=4)
    ax[0].imshow(img)
    ax[1].imshow(hist_equalization_result)
    ax[2].imshow(edgeEnhanced)
    ax[3].imshow(gray)
    plt.show() 

In [0]:

def rBrightness(image,ratio):
    hsv=cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    brightness=np.float64(hsv[:, :, 2])
    brightness=brightness*(1.0+np.random.uniform(-ratio,ratio))
    brightness[brightness>255]=255
    brightness[brightness<0]=0
    hsv[:, :, 2]=brightness
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
def rRotation(image, angle):
    if angle==0:
        return image
    angle=np.random.uniform(-angle,angle)
    rows,cols=image.shape[:2]
    size=rows,cols
    center=rows/2,cols/2
    scale=1.0
    rotation=cv2.getRotationMatrix2D(center,angle,scale)
    return cv2.warpAffine(image,rotation,size)
def rTranslation(image, translation):
    if translation==0:
        return 0
    rows,cols=image.shape[:2]
    size=rows,cols
    x=np.random.uniform(-translation,translation)
    y=np.random.uniform(-translation,translation)
    trans=np.float32([[1,0,x],[0,1,y]])
    return cv2.warpAffine(image,trans,size)
def rShear(image, shear):
    if shear==0:
        return image
    rows,cols=image.shape[:2]
    size=rows,cols
    left,right,top,bottom=shear,cols-shear,shear,rows-shear
    dx=np.random.uniform(-shear,shear)
    dy=np.random.uniform(-shear,shear)
    p1=np.float32([[left,top],[right,top],[left,bottom]])
    p2=np.float32([[left+dx,top],[right+dx,top+dy],[left,bottom+dy]])
    move=cv2.getAffineTransform(p1,p2)
    return cv2.warpAffine(image,move,size)
def agument_image(image,brightness,angle,translation,shear):
    image=rBrightness(image,brightness)
    image=rRotation(image,angle)
    image=rTranslation(image,translation)
    image=rShear(image,shear)
    return image

In [0]:
test_dir='/content/drive/My Drive/DL_PRJ/Traffic_Sign_Recognition_Detection/Source_Code/DataSet/Testing_DataSet/GTSRB/Final_Test/Images/'
labels_dir='/content/drive/My Drive/DL_PRJ/Traffic_Sign_Recognition_Detection/Source_Code/DataSet/Testing_DataSet/GT-final_test.csv'

In [0]:

def read_test_image():
  test_image_array=[]
  test_image_labels=[]
  test_Data_frame = pd.read_csv(labels_dir, delimiter=';')
  for _dir in os.listdir(test_dir):
    if _dir=='GT-final_test.test.csv':
      continue
    print("Directory: ", _dir)
    list_=test_Data_frame[test_Data_frame['Filename'] == _dir].index.tolist()
    dir_img_label=test_Data_frame.iloc[list_[0]]['ClassId']
    # label_df=pd.DataFrame([dir_img_label])
    # class_id=dir_img_label['ClassId']
    # print(dir_img_label)

    # img_path=os.path.join(_dir)
    # print(img_path)
    img = readImage(test_dir+_dir)
    img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
    test_image_array.append(img_features)
    test_image_labels.append(dir_img_label)
  return   test_image_array, test_image_labels



In [0]:
## Function to get csv path
def getCsvPath(train_path_dir):
     return os.path.join(train_path_dir,list(filter(lambda x: '.csv' in x, os.listdir(train_path_dir)))[0])

In [0]:
# Function to get category directory and corresponding data frame
def getDirAndDataFrame(_dir):
    train_category_dir= os.path.join(training_dataset_dir,_dir)
    train_csv_path= getCsvPath(train_category_dir)
    train_Data_frame = pd.read_csv(train_csv_path, delimiter=';')
    return train_category_dir, train_Data_frame

In [0]:
## Read image in path
def readImage(path):
    img= cv2.imread(path)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img= cv2.resize(img,(80,80))
    return img

In [0]:
def loadSampleImg(path):
    img=readImage(path)
    return img

In [0]:
## Training DataSet Directory
training_dataset_dir = '/content/drive/My Drive/DL_PRJ/Traffic_Sign_Recognition_Detection/Source_Code/DataSet/Training_DataSet/GTSRB/Final_Training/Images'

def LoadTrainDataSet():
    train_image_array=[]
    train_labels_array=[]

    for _dir in os.listdir(training_dataset_dir):
        print("Directory: ", _dir)
        # Get Directory for train with certain class ID and its associated dataframe
        train_category_dir, train_data_frame= getDirAndDataFrame(_dir)
        # Get ClassID for entire category
        dir_img_label=train_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)

            img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
            train_image_array.append(img_features)
            #Populate associated labels
            train_labels_array.append(dir_img_label)
            #### for agumentation 
            random_number_probability=random.uniform(0, 1)
            if random_number_probability>=0.5:
              agumented_imgae=agument_image(img,0.7,11,5,2)
              img_features_= np.array(FeatureExtraction(agumented_imgae))[:,:,np.newaxis]
              train_labels_array.append(dir_img_label)
              train_image_array.append(img_features_)
            
    train_image_array=np.stack(train_image_array, axis=0)
    return train_image_array, train_labels_array

In [28]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [29]:


import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
/device:GPU:0
Num GPUs Available:  1


In [0]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))

In [0]:
class CNN_Model(tf.keras.Model):
    def __init__(self):
        super(CNN_Model, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv_layer_1 = tf.keras.layers.Conv2D(filter_size,kernel_1, strides = 1, padding='same',activation = tf.nn.relu)
        self.conv_layer_2 = tf.keras.layers.Conv2D(64,kernel_size=(3, 3), strides = 1, padding='same',activation = tf.nn.relu)
        self.pool_layer_2 = tf.keras.layers.MaxPool2D(pooling)
        self.drop_2 =       tf.keras.layers.Dropout(dropout)
        self.conv_layer_3 = tf.keras.layers.Conv2D(64,kernel_2, strides = 1, padding='same', activation = tf.nn.relu)
        self.pool_layer_3 = tf.keras.layers.MaxPool2D(pooling)
        self.drop_3 =       tf.keras.layers.Dropout(dropout)
        self.conv_layer_4 = tf.keras.layers.Conv2D(filter_size,kernel_2, strides = 1, padding='same', activation = tf.nn.relu)
        self.pool_layer_4 = tf.keras.layers.MaxPool2D(pooling)
        # self.drop_4 =       tf.keras.layers.Dropout(dropout)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(hidden_layer, activation = tf.nn.relu)
        self.drop = tf.keras.layers.Dropout(dropout)
        self.final = tf.keras.layers.Dense(num_classes, activation = 'softmax')
    
    def call(self, input_tensor, training=False):
        x = self.conv_layer_1(input_tensor)
        x = self.conv_layer_2(x)
        x = self.pool_layer_2(x)
        x=  self.drop_2(x)
        x = self.conv_layer_3(x)
        x = self.pool_layer_3(x)  
        x=  self.drop_3(x)      
        x = self.conv_layer_4(x)
        # x = self.pool_layer_4(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.drop(x)
        x = self.final(x)
        return x

In [0]:
def train(TrainingDataSet, ValidationDataSet, model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):  

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            # Training DataSet
            for x_np, y_np in TrainingDataSet:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in ValidationDataSet:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [0]:
# Hyper-parameters tuning
kernel_1 = (5,5)
kernel_2 = (3,3)
pooling = (2,2)
filter_size = 32
hidden_layer = 64
dropout = 0.2
num_classes = 43

# Configurations of batch printing
print_every = 1000
num_epochs = 20
learning_rate = 1.5e-3

In [0]:
model = CNN_Model()

def model_init_fn():
    return CNN_Model()

In [0]:
def optimizer_init_fn():
    return tf.keras.optimizers.Adam(learning_rate) 

In [0]:
def saveCNNModel():
    model.save('cnn_model.model')

In [0]:
DataSetImages, DataSetLabels = LoadTrainDataSet()

Directory:  00042
Directory:  00041
Directory:  00039
Directory:  00038
Directory:  00037
Directory:  00040
Directory:  00035


In [1]:
test_image_array, test_image_labels=read_test_image()

NameError: ignored

In [77]:
image_train, image_valid,label_train, label_valid = train_test_split(DataSetImages,DataSetLabels,stratify=DataSetLabels,test_size=0.2,random_state=42)

NameError: ignored

In [0]:
print(image_train.shape)
TrainingDataSet= Dataset(image_train,np.array(label_train), batch_size=64, shuffle=True)
ValidationDataSet= Dataset(image_valid,np.array(label_valid), batch_size=64, shuffle=True)
for t, (x, y) in enumerate(TrainingDataSet):
    print(t, x.shape, y.shape)
    if t > 5: break

In [0]:
train(TrainingDataSet, ValidationDataSet, model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

In [0]:
model._set_inputs(image_train,np.array(label_train))
saveCNNModel()

In [0]:
saves_model_path='/content/drive/My Drive/DL_PRJ/Traffic_Sign_Recognition_Detection/Source_Code/Model/cnn_model.model'

In [0]:
def load_model():
  Model = tf.keras.models.load_model(saves_model_path)
  return Model

In [0]:
model= load_model()

In [0]:
prediction_y=model.predict(test_image_array)